# Graph Neutral Network
- **2024/25/1**: Önálló laboratórium 2
- Sági Benedek
- **Konzulens**: Unyi Dániel
- **Projekt és a célja**: A gráf neurális hálózatban a döntésekhez készítünk egy/több magyarázhatósági algoritmust, mely elmondja, hogy egy adott modell miért hozott ilyen döntést.




In [1]:
!pip uninstall torch
!pip install torch==2.4.0

Found existing installation: torch 2.5.0+cu121
Uninstalling torch-2.5.0+cu121:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-2.5.0+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.5.0+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 

In [2]:
import torch
!pip install torch_geometric
if torch.cuda.is_available():
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
else:
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.0/541.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.5/787.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 23.9 MB/s eta 0:00:00


In [71]:
from torch_geometric.data import Data
import torch_geometric

from torch_geometric import datasets

from torch_geometric.nn import GCNConv, SAGEConv, global_mean_pool,BatchNorm
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader

from torch_geometric.loader import NeighborLoader
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.loader import LinkLoader

import torch_sparse
import torch_geometric.transforms as T

from torch_geometric.utils import negative_sampling
from torch_geometric.utils import train_test_split_edges

import numpy as np

import wandb
import random

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Lépések:

##1.fázis
---
1. Adatok letöltése, előkészítése
Mivel a pytorch geometrich könyvtárnak van az az API-ja, amivel le lehet tölteni a kiválasztott adathalmazokat.
2. Adatok áttekintése
3. Modellek készítése különböző célokra
  * Csúcs szintű osztályozás
  * Gráf szintű osztályozás
  * Élprédikció

4. Eredmények elemzése
---
Nem ajánlott egyszerre összeset indítani, mivel akkor sokáig fog tartani, főleg a negyediknél.






### 1. Adatok letöltése

In [5]:
#WikipediaNetwork
dataset_WikipediaNetwork = datasets.WikipediaNetwork(root='data/WikipediaNetwork', name = 'chameleon')
#Amazon
dataset_Amazon = datasets.Amazon(root='data/Amazon', name = 'Computers')
#FacebookPagePage
dataset_FacebookPagePage = datasets.FacebookPagePage(root='data/FacebookPagePage')
#ZINC (https://pytorch-geometric.readthedocs.io/en/2.5.2/generated/torch_geometric.datasets.ZINC.html#torch_geometric.datasets.ZINC)
dataset_ENZYMES = datasets.TUDataset(root='data/TUDataset', name ='ENZYMES')
dataset_MNIST = datasets.GNNBenchmarkDataset(root='data/GNNBenchmarkDataset', name='MNIST')

Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Extracting data/GNNBenchmarkDataset/MNIST/raw/MNIST_v2.zip
Processing...
Done!


### 2. Adatok áttekintése

In [ ]:
def dataset_info(dataset):
  print(f'Dataset: {dataset}:')
  print('======================')
  print(f'Number of graphs: {len(dataset)}')
  print(f'Number of features: {dataset.num_features}')
  print(f'Number of classes: {dataset.num_classes}')
  data = dataset[0]  # Get the first graph object.
  print(data)
  print(f'Number of nodes: {data.num_nodes}')
  print(f'Number of edges: {data.num_edges}')
  print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
  print(f'Has isolated nodes: {data.has_isolated_nodes()}')
  print(f'Has self-loops: {data.has_self_loops()}')
  print(f'Is undirected: {data.is_undirected()}')

In [ ]:
dataset_info(dataset_WikipediaNetwork)
dataset_info(dataset_Amazon)
dataset_info(dataset_FacebookPagePage)

Dataset: WikipediaNetwork():
Number of graphs: 1
Number of features: 2325
Number of classes: 5
Data(x=[2277, 2325], edge_index=[2, 36101], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10])
Number of nodes: 2277
Number of edges: 36101
Average node degree: 15.85
Has isolated nodes: False
Has self-loops: True
Is undirected: False
Dataset: AmazonComputers():
Number of graphs: 1
Number of features: 767
Number of classes: 10
Data(x=[13752, 767], edge_index=[2, 491722], y=[13752])
Number of nodes: 13752
Number of edges: 491722
Average node degree: 35.76
Has isolated nodes: True
Has self-loops: False
Is undirected: True
Dataset: FacebookPagePage():
Number of graphs: 1
Number of features: 128
Number of classes: 4
Data(x=[22470, 128], edge_index=[2, 342004], y=[22470])
Number of nodes: 22470
Number of edges: 342004
Average node degree: 15.22
Has isolated nodes: False
Has self-loops: True
Is undirected: True


### 3. Modellek készítése

#### Csúcs szintű osztályozás
- Gráf konvolucios hálózat
Innen vettem ezt az alapvető kódot: https://colab.research.google.com/drive/14OvFnAXggxB8vM4e8vSURUp1TaKnovzX?usp=sharing#scrollTo=fmXWs1dKIzD8
- Ez alapján bovitettem: https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers
- A működését ennek segítségével értelmeztem: https://tkipf.github.io/graph-convolutional-networks/

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.conv4 = GCNConv(hidden_dim, output_dim)

    def  forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        return F.log_softmax(x, dim=1)

#### Élprédikáció

In [ ]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LinkPredictor, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim * 2, 1)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, edge_index):
        src, dst = edge_index
        z_src, z_dst = z[src], z[dst]
        return torch.sigmoid(self.fc(torch.cat([z_src, z_dst], dim=1)))

    def forward(self, data):
        z = self.encode(data.x, data.edge_index)
        return self.decode(z, data.edge_index)

#### Gráfszintu osztalyozas

In [66]:
class GCNGraphClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNGraphClassifier, self).__init__()
        # Define convolution layers with both SAGE and GCN layers
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.conv4 = GCNConv(hidden_dim, hidden_dim)

        # Batch normalization layers for stabilizing training
        self.bn1 = BatchNorm(hidden_dim)
        self.bn2 = BatchNorm(hidden_dim)
        self.bn3 = BatchNorm(hidden_dim)
        self.bn4 = BatchNorm(hidden_dim)

        # Fully connected layer for classification
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Forward pass with activation and batch normalization
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = F.relu(self.bn3(self.conv3(x, edge_index)))
        x = F.relu(self.bn4(self.conv4(x, edge_index)))

        # Global pooling
        x = global_mean_pool(x, batch)

        # Dropout and fully connected layer
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)


### 4. Eredmények és elemzése

In [87]:
batch_size = 256
num_neighbors = [10,10]
lr = 2e-3
hidden_dim = 64





In [13]:
def creatingMask(dataset, train_ratio=0.8, test_ratio = 0.2):
  data = dataset[0]
  num_nodes = data.num_nodes
  perm = torch.randperm(data.num_nodes)
  train_mask = perm[:int(data.num_nodes * train_ratio)]
  test_mask = perm[int(data.num_nodes * test_ratio):]
  data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
  data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
  data.train_mask[train_mask] = True
  data.test_mask[test_mask] = True
  return data

#### Csúcs szintű osztályozás

In [ ]:
def node_train_model(train_loader, test_loader, optimizer, model, device):
  def train():
      count=0
      model.train()
      total_loss = 0
      for batch in train_loader:
          batch = batch.to(device)
          optimizer.zero_grad()
          out = model(batch.x, batch.edge_index)
          criterion = CrossEntropyLoss()
          loss = criterion(out[:batch.batch_size], batch.y[:batch.batch_size])
          loss.backward()
          optimizer.step()
          total_loss += loss.item()
      return total_loss / len(train_loader)
  def test():
      model.eval()
      correct = 0
      total = 0  # Összes tesztadat száma
      for batch in test_loader:
          batch = batch.to(device)
          out = model(batch.x, batch.edge_index)
          pred = out.max(dim=1)[1]
          correct += pred.eq(batch.y).sum().item() # Kiszámolja, hogy abbol mennyi el lett találva az eq metódus segítségével
          total += batch.y.size(0)  # Összes minta száma
      return correct / total
  for epoch in range(10):
    loss = train()
    acc = test()
    #Elemzés
    print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')

In [ ]:
def nodeMainModel(data, num_classes, num_features):
  print(num_neighbors)
  print(batch_size)
  print(data.train_mask.shape)

  train_loader = NeighborLoader(data, num_neighbors=num_neighbors , batch_size=batch_size, input_nodes=data.train_mask)
  test_loader = NeighborLoader(data, num_neighbors=num_neighbors , batch_size=batch_size, input_nodes=data.test_mask)

  model = GCN(input_dim=num_features, hidden_dim=hidden_dim, output_dim=num_classes).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
  node_train_model(train_loader, test_loader, optimizer, model, device)

In [ ]:
#with masks
for dataset in [dataset_WikipediaNetwork]:
  print(dataset)
  data = dataset[0]
  data.train_mask = data.train_mask[:,0]
  data.test_mask = data.test_mask[:,0]
  nodeMainModel(data,dataset.num_classes, dataset.num_features)
#without masks
for dataset in [dataset_Amazon,dataset_FacebookPagePage]:
  print(dataset)
  nodeMainModel(creatingMask(dataset), dataset.num_classes, dataset.num_features)

WikipediaNetwork()
[10, 10]
128
torch.Size([2277])
Epoch 0, Loss: 1.5994, Test Accuracy: 0.2999
Epoch 1, Loss: 1.5477, Test Accuracy: 0.4459
Epoch 2, Loss: 1.4754, Test Accuracy: 0.4548
Epoch 3, Loss: 1.3654, Test Accuracy: 0.4802
Epoch 4, Loss: 1.2352, Test Accuracy: 0.4785
Epoch 5, Loss: 1.1273, Test Accuracy: 0.4874
Epoch 6, Loss: 1.0315, Test Accuracy: 0.4619
Epoch 7, Loss: 0.9806, Test Accuracy: 0.4625
Epoch 8, Loss: 0.9118, Test Accuracy: 0.4597
Epoch 9, Loss: 0.8523, Test Accuracy: 0.4691
AmazonComputers()
[10, 10]
128
torch.Size([13752])
Epoch 0, Loss: 1.2907, Test Accuracy: 0.7183
Epoch 1, Loss: 0.5991, Test Accuracy: 0.7791
Epoch 2, Loss: 0.5053, Test Accuracy: 0.8025
Epoch 3, Loss: 0.4599, Test Accuracy: 0.8088
Epoch 4, Loss: 0.4256, Test Accuracy: 0.8025
Epoch 5, Loss: 0.4079, Test Accuracy: 0.8059
Epoch 6, Loss: 0.3870, Test Accuracy: 0.8034
Epoch 7, Loss: 0.3759, Test Accuracy: 0.7980
Epoch 8, Loss: 0.3650, Test Accuracy: 0.7946
Epoch 9, Loss: 0.3450, Test Accuracy: 0.788

#### Élprédikáció

In [ ]:
def edge_train_model(train_loader, test_loader, optimizer, model, device):
    def train():
        total_loss = 0
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Kódolás
            z = model.encode(batch.x, batch.edge_index)

            # Pozitív élek dekódolása
            pos_pred = model.decode(z, batch.edge_index)

            # Negatív minták generálása
            neg_edge_index = negative_sampling(
                edge_index=batch.edge_index, num_nodes=batch.x.size(0), num_neg_samples=batch.edge_index.size(1)
            )

            # Negatív élek dekódolása
            neg_pred = model.decode(z, neg_edge_index)

            # Loss számítása: Pozitív és negatív minták összehasonlítása
            pos_loss = F.binary_cross_entropy(pos_pred.squeeze(), torch.ones(pos_pred.size(0)).to(device))
            neg_loss = F.binary_cross_entropy(neg_pred.squeeze(), torch.zeros(neg_pred.size(0)).to(device))

            # Teljes veszteség
            loss = pos_loss + neg_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test():
        model.eval()
        correct = 0
        total = 0  # Összes tesztadat száma

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)

                # Kódolás és predikció
                z = model.encode(batch.x, batch.edge_index)

                # Pozitív élek predikciója
                pos_pred = model.decode(z, batch.edge_index)

                # Negatív élek mintavételezése (azaz BSZ2 szempontjából komplementere)
                neg_edge_index = negative_sampling(
                    edge_index=batch.edge_index, num_nodes=batch.x.size(0), num_neg_samples=batch.edge_index.size(1)
                )
                neg_pred = model.decode(z, neg_edge_index)

                # Predikciók összehasonlítása
                pos_correct = (pos_pred > 0.5).sum().item()
                neg_correct = (neg_pred < 0.5).sum().item()

                correct += pos_correct + neg_correct
                #correct += pos_correct
                total += pos_pred.size(0) + neg_pred.size(0)
                #total += pos_pred.size(0)
        return correct / total

    for epoch in range(10):
        loss = train()
        acc = test()
        # Elemzés
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')


In [ ]:
def edgeMainModel(data,  num_classes, num_features, is_undirected):

  transform = RandomLinkSplit(is_undirected=is_undirected)
  train_data, val_data, test_data = transform(data)

  train_loader = LinkNeighborLoader(
      train_data,
      num_neighbors=num_neighbors,
      batch_size=batch_size,
      edge_label_index=train_data.edge_index,
      shuffle=True
  )

  test_loader = LinkNeighborLoader(
      test_data,
      num_neighbors=num_neighbors,
      batch_size=batch_size,
      edge_label_index=test_data.edge_index,
      shuffle=False
  )

  model = LinkPredictor(num_features, hidden_dim=hidden_dim).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  edge_train_model(train_loader, test_loader, optimizer, model, device)

In [ ]:
#with masks
for dataset in [dataset_WikipediaNetwork]:
  data = dataset[0]
  data.train_mask = data.train_mask[:,0]
  data.test_mask = data.test_mask[:,0]
  edgeMainModel(data, dataset.num_classes, dataset.num_features)
#without masks
for dataset in [dataset_Amazon, dataset_FacebookPagePage]:
  print(dataset)
  edgeMainModel(creatingMask(dataset),dataset.num_classes, dataset.num_features)

Epoch 0, Loss: 1.1222, Test Accuracy: 0.7421
Epoch 1, Loss: 0.9655, Test Accuracy: 0.6942
Epoch 2, Loss: 0.9431, Test Accuracy: 0.7832
Epoch 3, Loss: 0.9272, Test Accuracy: 0.7638
Epoch 4, Loss: 0.9219, Test Accuracy: 0.7901
Epoch 5, Loss: 0.9176, Test Accuracy: 0.7257
Epoch 6, Loss: 0.9121, Test Accuracy: 0.7905
Epoch 7, Loss: 0.9061, Test Accuracy: 0.8000
Epoch 8, Loss: 0.9062, Test Accuracy: 0.7833
Epoch 9, Loss: 0.9050, Test Accuracy: 0.7820
AmazonComputers()
Epoch 0, Loss: 0.7639, Test Accuracy: 0.8555
Epoch 1, Loss: 0.7043, Test Accuracy: 0.8580
Epoch 2, Loss: 0.6970, Test Accuracy: 0.8597
Epoch 3, Loss: 0.6940, Test Accuracy: 0.8585
Epoch 4, Loss: 0.6902, Test Accuracy: 0.8588
Epoch 5, Loss: 0.6891, Test Accuracy: 0.8595
Epoch 6, Loss: 0.6838, Test Accuracy: 0.8587
Epoch 7, Loss: 0.6793, Test Accuracy: 0.8615
Epoch 8, Loss: 0.6785, Test Accuracy: 0.8586
Epoch 9, Loss: 0.6770, Test Accuracy: 0.8603
FacebookPagePage()
Epoch 0, Loss: 0.7785, Test Accuracy: 0.8543
Epoch 1, Loss: 0.7

#### Gráfszintű osztályozás

In [94]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="GNN_project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "GNN",
    "dataset": "dataset_ENZYMES",
    "epochs": 171,
    }
)

wandb: Currently logged in as: 0322sagibenedek (0322sagibenedek-budapesti-m-szaki-s-gazdas-gtudom-nyi-eg). Use `wandb login --relogin` to force relogin


In [83]:
def graph_train_model(train_loader, test_loader, optimizer, criterion, model, device):
    def train():
      loss = 0
      model.train()
      for data in train_loader:  # Iterate in batches over the training dataset.
          data = data.to(device)
          out = model(data)  # Perform a single forward pass.
          loss = criterion(out, data.y)  # Compute the loss.
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          optimizer.zero_grad()  # Clear gradients.
          loss += loss.item()
      return loss / len(train_loader)
    def test():
        model.eval()
        correct = 0
        for data in test_loader:  # Iterate in batches over the training/test dataset.
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        return correct / len(test_loader.dataset)  # Derive ratio of correct predictions.


    for epoch in range(1, 171):
        loss = train()
        acc = test()
        print(f'Epoch: {epoch:03d},  Acc: {loss:.4f}, Loss: {acc:.4f}')
        wandb.log({"acc": acc, "loss": loss})


In [89]:
def graphMainModel(dataset,  num_classes, num_node_features):
  dataset = dataset_MNIST.shuffle()
  train_dataset = dataset[:150]
  test_dataset = dataset[150:]
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  model = GCNGraphClassifier(dataset.num_node_features, hidden_dim=hidden_dim, output_dim=dataset.num_classes).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  criterion = torch.nn.CrossEntropyLoss()
  graph_train_model(train_loader,test_loader,optimizer, criterion, model, device)


In [95]:
for dataset in [dataset_ENZYMES]:
  print(dataset)
  graphMainModel(dataset, dataset.num_classes, dataset.num_node_features)



ENZYMES(600)
Epoch: 001,  Acc: 4.7002, Loss: 0.0994
Epoch: 002,  Acc: 4.5566, Loss: 0.1134
Epoch: 003,  Acc: 4.4442, Loss: 0.1124
Epoch: 004,  Acc: 4.3469, Loss: 0.1124
Epoch: 005,  Acc: 4.2095, Loss: 0.1127
Epoch: 006,  Acc: 4.2140, Loss: 0.1145
Epoch: 007,  Acc: 4.1633, Loss: 0.1312
Epoch: 008,  Acc: 4.0490, Loss: 0.1894
Epoch: 009,  Acc: 4.0440, Loss: 0.1360
Epoch: 010,  Acc: 3.9950, Loss: 0.0994
Epoch: 011,  Acc: 3.9086, Loss: 0.0994
Epoch: 012,  Acc: 3.9330, Loss: 0.0994
Epoch: 013,  Acc: 3.9244, Loss: 0.1023
Epoch: 014,  Acc: 3.9468, Loss: 0.1041
Epoch: 015,  Acc: 3.7816, Loss: 0.1018
Epoch: 016,  Acc: 3.7704, Loss: 0.1007
Epoch: 017,  Acc: 3.7589, Loss: 0.1004
Epoch: 018,  Acc: 3.7904, Loss: 0.1006
Epoch: 019,  Acc: 3.7124, Loss: 0.1015
Epoch: 020,  Acc: 3.6731, Loss: 0.1024
Epoch: 021,  Acc: 3.6492, Loss: 0.1031
Epoch: 022,  Acc: 3.5786, Loss: 0.1034
Epoch: 023,  Acc: 3.5437, Loss: 0.1038
Epoch: 024,  Acc: 3.5859, Loss: 0.1041
Epoch: 025,  Acc: 3.5339, Loss: 0.1042
Epoch: 026, 

KeyboardInterrupt: 

In [96]:
wandb.finish()

acc,▁▁▁▁▁▃▃▄▄▄▄▄▃▄▄▆▆▆▄▃▅▅▆▃▃▂▃▄▃▃▄▃▄▇█▆▆▄▄▃
loss,███▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▃▂▃▃▂▂▂▂▂▁▁▁▁
acc,0.17758
loss,1.18941


## 2.fázis
--------
Folytatás
--------
